In [2]:
dmos_systems = ['S5G16', 'S5G13', 'S5G11', 'S5G8', 'S5G6', 'S15G11', 'S15G8', 'S15G6', 'S15G3', 'S25G6', 'S25G3', 'S25G1', 'S35G1', 'S45G1', 'ORI']
dmos_scores = [1.6560509554140128, 1.6928104575163399, 2.0194805194805197, 2.1575342465753424, 2.228187919463087, 2.4680851063829787, 2.675, 2.8701298701298703, 3.212328767123288, 3.2193548387096773, 3.643835616438356, 3.858064516129032, 4.3, 4.477707006369426, 4.726114649681529]
dmos_dict = dict(zip(dmos_systems, dmos_scores))

In [3]:
dmos_dict

{'S5G16': 1.6560509554140128,
 'S5G13': 1.6928104575163399,
 'S5G11': 2.0194805194805197,
 'S5G8': 2.1575342465753424,
 'S5G6': 2.228187919463087,
 'S15G11': 2.4680851063829787,
 'S15G8': 2.675,
 'S15G6': 2.8701298701298703,
 'S15G3': 3.212328767123288,
 'S25G6': 3.2193548387096773,
 'S25G3': 3.643835616438356,
 'S25G1': 3.858064516129032,
 'S35G1': 4.3,
 'S45G1': 4.477707006369426,
 'ORI': 4.726114649681529}

In [ ]:
import os
import torchaudio
from tqdm import tqdm
from torchmetrics.audio.pesq import PerceptualEvaluationSpeechQuality

# Initialize PESQ metric
pesq_metric = PerceptualEvaluationSpeechQuality(fs=16000, mode='nb')

# Directory containing subdirectories with WAV files
base_dir = '../test_samples_by_system'
reference_dir = os.path.join(base_dir, 'original')

# Dictionary to store mean PESQ scores for each subdirectory
mean_pesq_scores = {}

# Load reference WAV files
reference_wavs = {}
for file in os.listdir(reference_dir):
    if file.endswith('.wav'):
        file_path = os.path.join(reference_dir, file)
        waveform, sample_rate = torchaudio.load(file_path)
        reference_wavs[file] = waveform

In [11]:
# Iterate over each subdirectory
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    if os.path.isdir(subdir_path) and subdir != 'original':
        pesq_scores = []
        # Iterate over each WAV file in the subdirectory
        for file in tqdm(os.listdir(subdir_path)):
            file_path = os.path.join(subdir_path, file)
            file_name = file[:8]+'.wav'
            # Load the WAV file
            degraded_waveform, sample_rate = torchaudio.load(file_path)
            # Calculate PESQ score using the reference waveform
            pesq_score = pesq_metric(reference_wavs[file_name], degraded_waveform)
            pesq_scores.append(pesq_score.item())
        # Calculate mean PESQ score for the subdirectory
        if pesq_scores:
            mean_pesq_scores[subdir] = sum(pesq_scores) / len(pesq_scores)

mean_pesq_scores

100%|██████████| 50/50 [00:07<00:00,  6.37it/s]


{'SNR15-GD11d25': 2.304280676841736,
 'SNR25-GD1d25': 3.7990188598632812,
 'SNR15-GD3d75': 3.1559175157547,
 'SNR25-GD6d25': 3.270290899276733,
 'SNR15-GD6d25': 2.8429664278030398,
 'SNR45-GD1d25': 4.4097826957702635,
 'SNR15-GD8d75': 2.5103011417388914,
 'SNR5-GD13d75': 2.2606804513931276,
 'SNR5-GD16d25': 1.9089138746261596,
 'SNR25-GD3d75': 3.516825547218323,
 'SNR5-GD8d75': 2.8220131635665893,
 'SNR35-GD1d25': 4.187662181854248,
 'SNR5-GD11d25': 2.601540353298187,
 'SNR5-GD6d25': 3.0331207084655762}

In [18]:
# dmos_systems = ['SNR5-GD16d25', 'SNR5-GD13d75', 'SNR5-GD11d25', 'SNR5-GD8d75', 'SNR5-GD6d25', 
#                 'SNR15-GD11d25', 'SNR15-GD8d75', 'SNR15-GD6d25', 'SNR15-GD3d75', 'SNR25-GD6d25', 
#                 'SNR25-GD3d75', 'SNR25-GD1d25', 'SNR35-GD1d25', 'SNR45-GD1d25', 'ORI']
dmos_systems = ['S5G16', 'S5G13', 'S5G11', 'S5G8', 'S5G6', 'S15G11', 'S15G8', 'S15G6', 'S15G3', 'S25G6', 'S25G3', 'S25G1', 'S35G1', 'S45G1', 'ORI']

In [14]:
# Sort mean_pesq_scores by the keys (subdirectory names)
sorted_mean_pesq_scores = dict(sorted(mean_pesq_scores.items()))

# Print the sorted mean_pesq_scores along with dmos_systems
for system in dmos_systems:
    if system in sorted_mean_pesq_scores:
        print(f"{system}: {sorted_mean_pesq_scores[system]}")
    else:
        print(f"{system}: Not available")

SNR5-GD16d25: 1.9089138746261596
SNR5-GD13d75: 2.2606804513931276
SNR5-GD11d25: 2.601540353298187
SNR5-GD8d75: 2.8220131635665893
SNR5-GD6d25: 3.0331207084655762
SNR15-GD11d25: 2.304280676841736
SNR15-GD8d75: 2.5103011417388914
SNR15-GD6d25: 2.8429664278030398
SNR15-GD3d75: 3.1559175157547
SNR25-GD6d25: 3.270290899276733
SNR25-GD3d75: 3.516825547218323
SNR25-GD1d25: 3.7990188598632812
SNR35-GD1d25: 4.187662181854248
SNR45-GD1d25: 4.4097826957702635
ORI: Not available


In [16]:
pesq_score = pesq_metric(reference_wavs[file_name], reference_wavs[file_name])
print(pesq_score)

tensor(4.5486)


In [20]:
mean_pesq_scores['ORI'] = pesq_score.item()

In [21]:
mean_pesq_scores

{'SNR15-GD11d25': 2.304280676841736,
 'SNR25-GD1d25': 3.7990188598632812,
 'SNR15-GD3d75': 3.1559175157547,
 'SNR25-GD6d25': 3.270290899276733,
 'SNR15-GD6d25': 2.8429664278030398,
 'SNR45-GD1d25': 4.4097826957702635,
 'SNR15-GD8d75': 2.5103011417388914,
 'SNR5-GD13d75': 2.2606804513931276,
 'SNR5-GD16d25': 1.9089138746261596,
 'SNR25-GD3d75': 3.516825547218323,
 'SNR5-GD8d75': 2.8220131635665893,
 'SNR35-GD1d25': 4.187662181854248,
 'SNR5-GD11d25': 2.601540353298187,
 'SNR5-GD6d25': 3.0331207084655762,
 'ORI': 4.548638343811035}